In this notebook, we get to learn more about our data and conduct data cleaning & processing based on our project objective.

Our data source: 
* http://data.insideairbnb.com/united-states/ny/new-york-city/2018-10-03/visualisations/listings.csv
* http://data.insideairbnb.com/united-states/ny/new-york-city/2018-10-03/visualisations/reviews.csv
* http://web.mta.info/developers/data/nyct/subway/StationEntrances.csv
* https://data.cityofnewyork.us/Transportation/Bus-Stop-Shelters/qafz-7myz
* NRC-Emotion-Lexicon-v0.92-In105Languages-Nov2017Translations.csv

We used listings.csv, reviews.csv uploaded on 03 October, 2018 on insideairbnb website as our main data sets.
The other three csv files as our data source for added features. NRC-Emotion-Lexicon-v0.92-In105Languages-Nov2017Translations.csv contains 115 columns, 14182 rows. Each row includes a word in 105 kinds of languages and binary variables for 10 kinds of emotions.



In [12]:
import pandas as pd
import numpy as np

In [2]:
listing = pd.read_csv('listings.csv')

C:\Users\Baoyp\Anaconda3\envs\py36\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (43,87,88) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
listing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50041 entries, 0 to 50040
Data columns (total 96 columns):
id                                  50041 non-null int64
listing_url                         50041 non-null object
scrape_id                           50041 non-null int64
last_scraped                        50041 non-null object
name                                50018 non-null object
summary                             48185 non-null object
space                               34379 non-null object
description                         49610 non-null object
experiences_offered                 50041 non-null object
neighborhood_overview               29820 non-null object
notes                               19906 non-null object
transit                             31091 non-null object
access                              28928 non-null object
interaction                         27811 non-null object
house_rules                         29879 non-null object
thumbnail_url          

In [3]:
listing.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'access', 'interaction', 'house_rules',
       'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url',
       'host_id', 'host_url', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url',
       'host_picture_url', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms',

In [8]:
listing = listing[(listing['latitude'].notnull()) & (listing['longitude'].notnull())]

In [9]:
listing.rename(columns={"id": "listing_id"},inplace=True)

Standardize the format of zipcode feature

In [10]:
def fix_zip(input_zip):
    try:
        input_zip = int(float(input_zip))
    except:
        try:
            input_zip = int(input_zip.split('-')[0])
        except:
            return np.NaN
    if input_zip < 10000 or input_zip > 12000:
        return np.NaN
    return str(input_zip)

In [13]:
listing['zipcode']=listing['zipcode'].apply(fix_zip)
listing = listing[listing['zipcode'].notnull()]
listing['zipcode'].unique()

array(['10026', '11218', '10018', '11206', '10027', '11238', '10029',
       '10016', '11216', '10019', '10025', '10010', '10002', '11215',
       '10014', '11211', '11205', '10011', '11217', '10009', '10034',
       '10003', '10031', '11222', '11237', '11226', '10024', '11225',
       '11101', '11221', '11434', '11249', '10023', '11230', '11231',
       '10036', '10035', '11233', '10032', '11377', '11201', '10466',
       '10001', '10128', '10040', '11234', '11367', '10039', '10028',
       '11104', '10301', '10452', '10006', '11385', '10021', '10075',
       '11379', '10030', '11105', '10012', '10044', '10022', '10013',
       '11103', '10473', '10475', '10282', '10468', '11102', '10038',
       '11213', '10033', '11427', '11693', '10004', '11375', '10470',
       '10453', '10469', '11208', '10106', '11207', '10451', '11235',
       '11209', '10314', '11204', '10305', '11232', '11418', '10017',
       '11372', '11373', '11355', '11412', '11203', '11374', '11106',
       '11220', '103

Merge text mining results into mega dataset.

In [14]:
review = pd.read_csv('list_review_rating.csv')

In [15]:
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23281 entries, 0 to 23280
Data columns (total 12 columns):
listing_id             23281 non-null int64
Fear                   23281 non-null float64
Trust                  23281 non-null float64
Negative               23281 non-null float64
Positive               23281 non-null float64
Joy                    23281 non-null float64
Disgust                23281 non-null float64
Anticipation           23281 non-null float64
Anger                  23281 non-null float64
Sadness                23281 non-null float64
Surprise               23281 non-null float64
customer_experience    23281 non-null float64
dtypes: float64(11), int64(1)
memory usage: 2.1 MB


In [16]:
final = pd.merge(listing, review, on = 'listing_id', how='inner')

Remove unnecessary features from the mega dataset.

In [17]:
final.drop(['scrape_id', 'last_scraped','experiences_offered','notes', 'transit', 'access', 'interaction',
       'house_rules', 'thumbnail_url', 'medium_url','xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'market','is_location_exact','smart_location', 'country_code', 'country', 'is_location_exact', 'property_type',
           'accommodates','bed_type',  'security_deposit',
       'cleaning_fee', 'guests_included', 'extra_people', 'minimum_nights',
       'maximum_nights', 'calendar_updated', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'calendar_last_scraped', 'number_of_reviews',
       'first_review', 'last_review','requires_license',
       'license', 'jurisdiction_names', 'instant_bookable',
       'is_business_travel_ready', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification',
       'calculated_host_listings_count', 'reviews_per_month'],axis=1,inplace=True)

In [18]:
final.head()

,listing_id,listing_url,name,summary,space,description,neighborhood_overview,picture_url,host_response_time,host_response_rate,...,Trust,Negative,Positive,Joy,Disgust,Anticipation,Anger,Sadness,Surprise,customer_experience
0,2515,https://www.airbnb.com/rooms/2515,Stay at Chez Chic budget room #1,Step into our artistic spacious apartment and ...,-PLEASE BOOK DIRECTLY. NO NEED TO SEND A REQUE...,Step into our artistic spacious apartment and ...,NaN,https://a0.muscache.com/im/pictures/d0489e42-4...,NaN,NaN,...,0.015873,0.015873,0.039683,0.015873,0.000000,0.007937,0.007937,0.007937,0.000000,0.357143
1,2539,https://www.airbnb.com/rooms/2539,Clean & quiet apt home by the park,Renovated apt home in elevator building.,"Spacious, renovated, and clean apt home, one b...",Renovated apt home in elevator building. Spaci...,Close to Prospect Park and Historic Ditmas Park,https://a0.muscache.com/im/pictures/3949d073-a...,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2595,https://www.airbnb.com/rooms/2595,Skylit Midtown Castle,"Find your romantic getaway to this beautiful, ...","- Spacious (500+ft²), immaculate and nicely fu...","Find your romantic getaway to this beautiful, ...",Centrally located in the heart of Manhattan ju...,https://a0.muscache.com/im/pictures/f028bdf9-e...,within a day,80%,...,0.000000,0.000000,0.083333,0.027778,0.000000,0.027778,0.000000,0.027778,0.027778,0.972222
3,3330,https://www.airbnb.com/rooms/3330,++ Brooklyn Penthouse Guestroom ++,"This is a spacious, clean, furnished master be...","Room Features: - clean, hardwood floors - 2 la...","This is a spacious, clean, furnished master be...",Location is GREAT!! Right off the L train in t...,https://a0.muscache.com/im/pictures/41842659/5...,NaN,NaN,...,0.039823,0.000000,0.061947,0.030973,0.004425,0.048673,0.008850,0.004425,0.008850,1.150442
4,3831,https://www.airbnb.com/rooms/3831,Cozy Entire Floor of Brownstone,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Greetings! We own a double-duplex brownst...,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Just the right mix of urban center and local n...,https://a0.muscache.com/im/pictures/e49999c2-9...,within an hour,100%,...,0.028926,0.012397,0.033058,0.024793,0.002066,0.024793,0.004132,0.014463,0.012397,0.650826


## Merge transportation related features into mega dataset.

In [19]:
traffic = pd.read_csv('listing_new.csv')
traffic = traffic.drop('Unnamed: 0',axis=1)

In [20]:
traffic.rename(columns={"id": "listing_id"},inplace=True)

In [21]:
traffic.head()

,listing_id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,dis_to_closest_subway,closest_subway_station,dis_to_closest_bus,closest_bus_station
0,2515,Stay at Chez Chic budget room #1,2758,Steph,Manhattan,Harlem,40.799205,-73.953676,Private room,59,2,173,2018-09-18,1.42,3,310,0.102983,110th St-Central Park North,0.073829,"('MN0836', 'CENTRAL PARK NORTH')"
1,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.647486,-73.972370,Private room,149,1,8,2018-07-17,0.23,8,360,0.306068,Fort Hamilton Parkway,0.104210,"('BR02424', 'CONEY ISLAND AVENUE')"
2,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.753621,-73.983774,Entire home/apt,225,1,36,2018-10-01,0.33,2,345,0.101302,5th Av,0.139760,"('MN01685', '5 AVENUE')"
3,3330,++ Brooklyn Penthouse Guestroom ++,4177,Jbee,Brooklyn,Williamsburg,40.708558,-73.942362,Private room,70,5,34,2018-09-16,0.32,3,271,0.151482,Montrose Av,0.095851,"('BR02884', 'MONTROSE AVENUE')"
4,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.809018,-73.941902,Private room,150,3,0,NaN,NaN,1,365,0.219138,125th St,0.128132,"('MN01330', 'LENOX AVENUE')"


In [22]:
df = pd.merge(final, traffic, on = 'listing_id', how='inner')

In [23]:
df.head()

,listing_id,listing_url,name_x,summary,space,description,neighborhood_overview,picture_url,host_response_time,host_response_rate,...,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,dis_to_closest_subway,closest_subway_station,dis_to_closest_bus,closest_bus_station
0,2515,https://www.airbnb.com/rooms/2515,Stay at Chez Chic budget room #1,Step into our artistic spacious apartment and ...,-PLEASE BOOK DIRECTLY. NO NEED TO SEND A REQUE...,Step into our artistic spacious apartment and ...,NaN,https://a0.muscache.com/im/pictures/d0489e42-4...,NaN,NaN,...,2,173,2018-09-18,1.42,3,310,0.102983,110th St-Central Park North,0.073829,"('MN0836', 'CENTRAL PARK NORTH')"
1,2539,https://www.airbnb.com/rooms/2539,Clean & quiet apt home by the park,Renovated apt home in elevator building.,"Spacious, renovated, and clean apt home, one b...",Renovated apt home in elevator building. Spaci...,Close to Prospect Park and Historic Ditmas Park,https://a0.muscache.com/im/pictures/3949d073-a...,NaN,NaN,...,1,8,2018-07-17,0.23,8,360,0.306068,Fort Hamilton Parkway,0.104210,"('BR02424', 'CONEY ISLAND AVENUE')"
2,2595,https://www.airbnb.com/rooms/2595,Skylit Midtown Castle,"Find your romantic getaway to this beautiful, ...","- Spacious (500+ft²), immaculate and nicely fu...","Find your romantic getaway to this beautiful, ...",Centrally located in the heart of Manhattan ju...,https://a0.muscache.com/im/pictures/f028bdf9-e...,within a day,80%,...,1,36,2018-10-01,0.33,2,345,0.101302,5th Av,0.139760,"('MN01685', '5 AVENUE')"
3,3330,https://www.airbnb.com/rooms/3330,++ Brooklyn Penthouse Guestroom ++,"This is a spacious, clean, furnished master be...","Room Features: - clean, hardwood floors - 2 la...","This is a spacious, clean, furnished master be...",Location is GREAT!! Right off the L train in t...,https://a0.muscache.com/im/pictures/41842659/5...,NaN,NaN,...,5,34,2018-09-16,0.32,3,271,0.151482,Montrose Av,0.095851,"('BR02884', 'MONTROSE AVENUE')"
4,3831,https://www.airbnb.com/rooms/3831,Cozy Entire Floor of Brownstone,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Greetings! We own a double-duplex brownst...,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Just the right mix of urban center and local n...,https://a0.muscache.com/im/pictures/e49999c2-9...,within an hour,100%,...,1,213,2018-09-12,4.36,1,149,0.273505,Classon Av,0.040454,"('BR02441', 'GATES AVENUE')"


In [56]:
df.columns

Index(['listing_id', 'listing_url', 'name_x', 'summary', 'space',
       'description', 'neighborhood_overview', 'picture_url',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'neighbourhood_x', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode',
       'latitude_x', 'longitude_x', 'room_type_x', 'bathrooms', 'bedrooms',
       'beds', 'amenities', 'square_feet', 'price_x', 'weekly_price',
       'monthly_price', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'Fear', 'Trust', 'Negative', 'Positive', 'Joy',
       'Disgust', 'Anticipation', 'Anger', 'Sadness', 'Surprise',
       'customer_experience', 'name_y', 'host_id', 'host_name',
       'neighbourhood_group', 'neighbourhood_y', 'latitude_y', 'longitude_y',
       'room_type_y', 'pri

## Drop some duplicated columns from new mega dataset

In [24]:
df.drop([ 'name_y', 'host_id', 'host_name',
       'neighbourhood_group','neighbourhood_y', 'latitude_y', 'longitude_y',
       'room_type_y', 'price_y', 'minimum_nights', 'number_of_reviews',
       'last_review', 'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'], axis = 1,inplace=True)
df.drop(['host_response_time','host_response_rate','host_acceptance_rate'],axis=1,inplace=True)

In [27]:
df.to_csv('final.csv')

## Read back mega dataset with new restaurant feature and re-format price column by Regex

In [38]:
df = pd.read_csv('final_aftercount restaurant.csv')
df = df.drop('Unnamed: 0',axis=1)

C:\Users\Baoyp\Anaconda3\envs\py36\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [39]:
df = df.drop(['neighbourhood_x',],axis=1)

In [40]:
df.rename(columns = {'name_x':'name','neighbourhood_cleansed':'neighbourhood','neighbourhood_group_cleansed':'neighbourhood_group','latitude_x':'latitude', 
                     'longitude_x':'longtitude','room_type_x':'room_type','price_x':'price'},inplace=True)

In [41]:
df.price.head()

0     $59.00
1    $149.00
2    $225.00
3     $70.00
4     $89.00
Name: price, dtype: object

In [42]:
from re import sub
df.price = df.price.apply(lambda x: float(sub(r'[^\d.]', '', x)))

In [43]:
df.price.head()

0     59.0
1    149.0
2    225.0
3     70.0
4     89.0
Name: price, dtype: float64

In [44]:
df = df[df.price<500]

In [50]:
# Get rid of rows whose important features including null values
df = df[~df['bedrooms'].isnull()]
df = df[~df['bathrooms'].isnull()]
df = df[~df['review_scores_cleanliness'].isnull()]
df = df[~df['review_scores_checkin'].isnull()]
df = df[~df['review_scores_communication'].isnull()]
df = df[~df['customer_experience'].isnull()]
df = df[~df['dis_to_closest_subway'].isnull()]
df = df[~df['dis_to_closest_bus'].isnull()]
df = df[~df['num_restaurant'].isnull()]

In [49]:
# Dataset for visualization
df.to_csv('database_1.csv')

## Add num_review feature into mega dataset

In [59]:
df2 = pd.read_csv('reviews.csv')
#df2 = df2.drop('Unnamed: 0',axis=1)

In [60]:
review_num = df2.groupby('listing_id').size()
review_num=pd.DataFrame(review_num)

In [61]:
review_num.rename(columns={0:'review_num'},inplace=True)

In [62]:
df_final=pd.merge(df,review_num, on='listing_id',how = 'left')

In [63]:
df_final.to_csv('database_3.csv')

In [65]:
print(df_final.review_num.max())
print(df_final.review_num.min())
print(df_final.review_num.mode())

556
1
0    2
dtype: int64


## Normalize columns in dataframe

In [51]:
df['review_scores_cleanliness']=(df['review_scores_cleanliness']-np.mean(df['review_scores_cleanliness']))/np.std(df['review_scores_cleanliness'])

In [52]:
df['review_scores_checkin'] = (df['review_scores_checkin']-np.mean(df['review_scores_checkin']))/np.std(df['review_scores_checkin'])

In [53]:
df['review_scores_communication'] = (df['review_scores_communication']-np.mean(df['review_scores_communication']))/np.std(df['review_scores_communication'])

In [54]:
df['customer_experience'] = (df['customer_experience']-np.mean(df['customer_experience']))/np.std(df['customer_experience'])

In [55]:
df['num_restaurant'] = (df['num_restaurant']-np.mean(df['num_restaurant']))/np.std(df['num_restaurant'])

In [58]:
# Dataset for our recommendation system
df.to_csv('database_2.csv')